In [7]:
!pip install keras_tuner

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import keras_tuner as kt

def build_model(hp):
    model = models.Sequential()
    
    # Input layer (assume image size 48x48x1 for grayscale)
    model.add(layers.Input(shape=(48, 48, 1)))
    
    # First convolutional block
    model.add(layers.Conv2D(
        filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
        activation='relu',
        padding='same'
    ))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Second convolutional block
    model.add(layers.Conv2D(
        filters=hp.Int('conv_2_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
        activation='relu',
        padding='same'
    ))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten and Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(
        units=hp.Int('dense_units', min_value=64, max_value=256, step=64),
        activation='relu'
    ))
    model.add(layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output layer (7 classes, for example)
    model.add(layers.Dense(7, activation='softmax'))
    
    # Compile the model with a hyperparameter for learning rate
    model.compile(
        optimizer=optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [ ]:
# Create a tuner object
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',  # You can choose other objectives, e.g., val_loss
    max_trials=10,             # Number of different hyperparameter combinations to try
    executions_per_trial=1,    # How many models to build and fit for each trial (use >1 for more reliable estimates)
    directory='hyperparam_tuning',
    project_name='emotion_cnn'
)

# Print a summary of the search space
tuner.search_space_summary()

# Assume you have training and validation data generators or numpy arrays:
# For example, if using ImageDataGenerator:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'images/train/',
    target_size=(48,48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'images/test/',
    target_size=(48,48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

# Run the hyperparameter search
tuner.search(train_generator,
             validation_data=val_generator,
             epochs=20,
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])


Trial 7 Complete [00h 19m 56s]
val_accuracy: 0.5222903490066528

Best val_accuracy So Far: 0.5533574819564819
Total elapsed time: 01h 52m 45s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
96                |128               |conv_1_filters
3                 |5                 |conv_1_kernel
64                |64                |conv_2_filters
5                 |5                 |conv_2_kernel
64                |256               |dense_units
0.2               |0.2               |dropout_rate
0.0047338         |0.00023557        |learning_rate

Epoch 1/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 77s 85ms/step - accuracy: 0.2625 - loss: 1.8506 - val_accuracy: 0.3546 - val_loss: 1.6102
Epoch 2/20
514/898 ━━━━━━━━━━━━━━━━━━━━ 30s 79ms/step - accuracy: 0.3684 - loss: 1.6120